# Overview

In this notebook we are going to create a SparkContext object.

It assumes you have already read the following notebooks:
- [Install Apache Spark Prerequisites](Install%20Apache%20Spark%20Prerequisites.ipynb)
- [Running Apache Spark On Kubernetes](Running%20Apache%20Spark%20On%20Kubernetes.ipynb)

The instructions are basically the same as ...

## Adjenda
1. Configure Kubernetes Cluster For Spark
2. Install the Kubectl CLI for Kubernetes
3. Set Environment variables
4. Create SparKConf
5. Create SparkContext
6. Cleanup Spark and Kubernetes
7. Package This As A Helper Module

## 1. Configure Kubernetes Cluster For Spark
In order for our kubernetes cluster to successfully run a spark cluster we need to do a few things:
1. Configure RBAC - We will need to set permissions so that our jupyter notebook and spark components have the appropriate permissions.
2. Build containers - We will need to build the contaienrs which host our spark cluster nodes.

## 1.1. Configure Kubernetes RBAC

## 1.2. Build Spark Containers For Kubernetes

# 2. Install and Configure Kubectl
Kubectl is the CLI for kubernetes. It will allow our jupyter notebook to connect to the kubernetes cluster and spin up containers to run our Spark work.

## 2.1. Install Kubectl
There are a number of ways to install kubectl. The easiest and fully featured way is to use the chocolatey installation process.

https://kubernetes.io/docs/tasks/tools/install-kubectl-windows/#install-on-windows-using-chocolatey-or-scoop

In [1]:
! kubectl version --client

Client Version: version.Info{Major:"1", Minor:"21", GitVersion:"v1.21.0", GitCommit:"cb303e613a121a29364f75cc67d3d580833a7479", GitTreeState:"clean", BuildDate:"2021-04-08T16:31:21Z", GoVersion:"go1.16.1", Compiler:"gc", Platform:"windows/amd64"}


## 2.2. Configure Kubectl 

In [2]:
! cd %USERPROFILE% & mkdir .kube 2> NUL

Create the kubeconfi file... We can copy it from the master


! kubectl cluster-info

In [3]:
! kubectl get node

NAME                           STATUS   ROLES                  AGE   VERSION
os004k8-master001.foobar.com   Ready    control-plane,master   21d   v1.21.0
os004k8-worker001.foobar.com   Ready    <none>                 21d   v1.21.0
os004k8-worker002.foobar.com   Ready    <none>                 21d   v1.21.0
os004k8-worker003.foobar.com   Ready    <none>                 21d   v1.21.0


# 3. Set Environment Variables
We can use the os package to set environment variables

## 3.1. Set SPARK_HOME variable
This variable configures our system to understand where spark is installed.

In [4]:
import os

In [5]:
os.environ['SPARK_HOME'] = "c:\\spark\\spark-3.1.1-bin-hadoop2.7"

In [6]:
print(os.environ['SPARK_HOME'])

c:\spark\spark-3.1.1-bin-hadoop2.7


## 3.2. Run findspark.init() to add Spark to PATH
PySpark isn't on sys.path by default, but that doesn't mean it can't be used as a regular library. You can address this by either symlinking pyspark into your site-packages, or adding pyspark to sys.path at runtime. findspark does the latter.

https://github.com/minrk/findspark

In [7]:
import findspark
findspark.init()

In [8]:
# Print the PATH variable to show the spark directory is set
import sys
print(sys.path)

['c:\\spark\\spark-3.1.1-bin-hadoop2.7\\python', 'c:\\spark\\spark-3.1.1-bin-hadoop2.7\\python\\lib\\py4j-0.10.9-src.zip', 'c:\\program files\\python36\\python36.zip', 'c:\\program files\\python36\\DLLs', 'c:\\program files\\python36\\lib', 'c:\\program files\\python36', '', 'c:\\program files\\python36\\lib\\site-packages', 'c:\\program files\\python36\\lib\\site-packages\\win32', 'c:\\program files\\python36\\lib\\site-packages\\win32\\lib', 'c:\\program files\\python36\\lib\\site-packages\\Pythonwin', 'c:\\program files\\python36\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\Administrator\\.ipython']


## 3.3. Set PYSPARK_PYTHON variable
This variable configures spark to understand where python is installed on the spark nodes. Recall, these are the linux containers we built earlier. By default, the local windows file path may be set, but this will not work. If improperly confiugred we may see an error like this one:
```
Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 0.0 failed 4 times, most recent failure: Lost task 2.3 in stage 0.0 (TID 17) (10.36.0.2 executor 1): java.io.IOException: Cannot run program "c:\program files\python36\python.exe": error=2, No such file or directory
```
We need to set this variable equal to path of python on the container.

In [9]:
os.environ['PYSPARK_PYTHON'] = "/usr/bin/python3"

In [10]:
print(os.environ['PYSPARK_PYTHON'])

/usr/bin/python3


# 4. Create SparKConf Object

In [11]:
import pyspark

In [12]:
# Set some vars to specify where the kubernetes master is
kubernetes_master_ip = "15.4.7.11"
kubernetes_master_port = "6443"
spark_master_url = "k8s://https://{0}:{1}".format(kubernetes_master_ip, kubernetes_master_port)

In [13]:
# Determine the ip address of the machine
import netifaces
import re
nic_uuid = netifaces.gateways()['default'][netifaces.AF_INET][1]
nic_details = netifaces.ifaddresses(nic_uuid)
ip_address = None
for i, nic_detail, in nic_details.items():
    if all([key in nic_detail[0].keys() for key in ["addr", "netmask", "broadcast"]]):
        if re.match("([0-9]+\\.)+", nic_detail[0]["addr"]):
            ip_address = nic_detail[0]["addr"]
            break
print("The ip was detected as: {0}".format(ip_address))

The ip was detected as: 15.1.1.23


In [14]:
# Wire up the SparkConf object
sparkConf = pyspark.SparkConf()
sparkConf.setMaster(spark_master_url)

sparkConf.setAppName("spark-jupyter-win")

sparkConf.set("spark.submit.deploy.mode", "cluster")
sparkConf.set("spark.kubernetes.container.image", "tschneider/pyspark:v2") 
sparkConf.set("spark.kubernetes.namespace", "spark")
sparkConf.set("spark.kubernetes.pyspark.pythonVersion", "3")
sparkConf.set("spark.kubernetes.authenticate.driver.serviceAccountName", "spark-sa")
sparkConf.set("spark.kubernetes.authenticate.serviceAccountName", "spark-sa")

sparkConf.set("spark.executor.instances", "3")
sparkConf.set("spark.executor.cores", "2")
sparkConf.set("spark.executor.memory", "1024m")
sparkConf.set("spark.driver.memory", "1024m")

# If we are not using a hostname registered with a dns server, we need to set this parameter
sparkConf.set("spark.driver.host", ip_address)

In [15]:
sparkConf.getAll()

dict_items([('spark.master', 'k8s://https://15.4.7.11:6443'), ('spark.app.name', 'spark-jupyter-win'), ('spark.submit.deploy.mode', 'cluster'), ('spark.kubernetes.container.image', 'tschneider/pyspark:v2'), ('spark.kubernetes.namespace', 'spark'), ('spark.kubernetes.pyspark.pythonVersion', '3'), ('spark.kubernetes.authenticate.driver.serviceAccountName', 'spark-sa'), ('spark.kubernetes.authenticate.serviceAccountName', 'spark-sa'), ('spark.executor.instances', '3'), ('spark.executor.cores', '2'), ('spark.executor.memory', '1024m'), ('spark.driver.memory', '1024m'), ('spark.driver.host', '15.1.1.23')])

# 5. Create SparkContext

In [16]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()
sc = spark.sparkContext

We can look at kubernetes to see that out worker nodes were created.

In [17]:
! kubectl -n spark get pod

NAME                                        READY   STATUS    RESTARTS   AGE
spark-jupyter-win-454aa9797d84bef5-exec-1   1/1     Running   0          12s
spark-jupyter-win-454aa9797d84bef5-exec-2   1/1     Running   0          11s
spark-jupyter-win-454aa9797d84bef5-exec-3   1/1     Running   0          11s


# 6. Cleanup Spark Cluster On Kubernetes

In [29]:
sc.stop()

In [30]:
! kubectl -n spark get pod

NAME                                        READY   STATUS        RESTARTS   AGE
spark-jupyter-win-454aa9797d84bef5-exec-1   1/1     Terminating   0          63s
spark-jupyter-win-454aa9797d84bef5-exec-2   1/1     Terminating   0          62s
spark-jupyter-win-454aa9797d84bef5-exec-3   1/1     Terminating   0          62s


# 7. Package This As A Helper Module

I have packaged the code above into a helper module. We can include this module as a way to have this code execute in a neat and standard way. Simply execute the following in a cell:

```
include create_spark_context.py
```